In [1]:
import requests, re, json, datetime
import os, sys
from bs4 import BeautifulSoup as bs
from urllib.parse import urljoin
import uuid
from hashlib import md5
import dateparser
import pandas as pd


import ScrapeBakery as sb

In [2]:
now = datetime.datetime.now().timestamp()
a_day = 86400
days_since_start=1
start_timestamp = now-(a_day * days_since_start)
page = sb.IdeaPage("https://www.halfbakery.com/idea/Autoweeeebile", start_timestamp)

In [3]:
first_time=False
# This makes use of a search url, returning all the ideas posted in some time period (defaulted to a day)
a_day = 86400
if first_time == True:
    days_since_start = (datetime.datetime.now() - datetime.datetime(2020,10,17) ).days
    
    start_timestamp = now-(a_day * days_since_start)
    guess_m = 500
    t_minus = int(now - start_timestamp)
    urls = ["https://www.halfbakery.com/view/ftm=r{t_minus}:s=Qr:d=irq:dn={m}:ds=0:n=Today_27s_20Notions:i=A_20list_20of_20todays_20ideas_20and_20annotations:t=Today_27s_20Notions".format(m=guess_m,t_minus=t_minus),
            "https://www.halfbakery.com/view/ftm=r{t_minus}:s=Qr:d=irq:do=100:dn=100:ds=0:n=Today_27s_20Notions:i=A_20list_20of_20todays_20ideas_20and_20annotations:t=Today_27s_20Notions".format(t_minus=t_minus),
            "https://www.halfbakery.com/view/ftm=r{t_minus}:s=Qr:d=irq:do=200:dn=100:ds=0:n=Today_27s_20Notions:i=A_20list_20of_20todays_20ideas_20and_20annotations:t=Today_27s_20Notions".format(t_minus=t_minus)]
else:
    start_timestamp = now-(a_day * 1)
    guess_m = 100
    t_minus = int(now - start_timestamp)
    urls = ["https://www.halfbakery.com/view/ftm=r{t_minus}:s=Qr:d=irq:dn={m}:ds=0:n=Today_27s_20Notions:i=A_20list_20of_20todays_20ideas_20and_20annotations:t=Today_27s_20Notions".format(m=guess_m,t_minus=t_minus)]
    

data_harvest = sb.HarvestLinks(urls, start_timestamp)
print("Number of unique links to follow: {l}".format(l=len(data_harvest.fetch_links)))
data_harvest.fetch_data()

Number of unique links to follow: 14
Processing...
 14 pages to collect
Read 10 of 14...

Complete read 14 of 14


In [4]:
ds = sb.DataStore(trash=True)

Building new schema any existing content will be lost.
no such view: latest_user_content


In [5]:
ds.query_to_recordset("select tbl_name from sqlite_master")

[{'tbl_name': 'idea_fetch'},
 {'tbl_name': 'anno_fetch'},
 {'tbl_name': 'link_fetch'},
 {'tbl_name': 'fresh_ideas'},
 {'tbl_name': 'latest_user_content'}]

In [6]:
for idea in data_harvest.contents:
    ds.save_idea(idea)

In [7]:
pd.DataFrame(ds.query_to_recordset("select * from fresh_ideas")).sort_values(by="url")

,url,fetch_id,fetch_date
6,https://www.halfbakery.com/idea/10_20Gbps_20et...,b5f8a8ea-c334-4000-b533-78b02a990019,1.608400e+09
5,https://www.halfbakery.com/idea/Autoweeebile_20II,42742c2a-aa0d-4f46-ac2b-519b8e1e945a,1.608400e+09
13,https://www.halfbakery.com/idea/Autoweeebile_2...,6137360a-9def-432c-9243-cc6d18fdef70,1.608400e+09
0,https://www.halfbakery.com/idea/Autoweeeebile,0ecb1049-854f-44c9-8348-fe1ce8b5f10d,1.608400e+09
8,https://www.halfbakery.com/idea/Colouring_20Co...,c4c71029-f1e6-4d30-819a-0ba5d70dd406,1.608400e+09
7,https://www.halfbakery.com/idea/Completely_20R...,06c313ae-871e-494a-b0f5-70153e4c3b2b,1.608400e+09
11,https://www.halfbakery.com/idea/Days_20Since_2...,6867f277-7644-46ac-a5b8-ee4ee2dbed8e,1.608400e+09
2,https://www.halfbakery.com/idea/F_fcrst_20annu...,8e484517-6281-4da1-baf0-9bfc2aec45be,1.608400e+09
10,https://www.halfbakery.com/idea/Forget-Me-Do,e6e07f9a-9607-4eef-a55b-42a037f3882a,1.608400e+09
4,https://www.halfbakery.com/idea/Merry_20Christmas,cba958e5-13fa-485b-bb9a-8c1fb872fd0e,1.608400e+09


In [8]:
pd.DataFrame(ds.query_to_recordset("select (select count(*) from fresh_ideas) fresh, (select count(*) from idea_fetch) raw"))

,fresh,raw
0,14,14


In [11]:
pd.DataFrame(ds.query_to_recordset("select * from latest_user_content"))

,url,fetch_id,date,user,ctype,text
0,https://www.halfbakery.com/idea/Autoweeeebile,0ecb1049-854f-44c9-8348-fe1ce8b5f10d,NaN,ixnaum,idea,Imagine you are driving along a familiar highw...
1,https://www.halfbakery.com/idea/light,53334179-ca9a-4ff7-b0dd-dc3e4a0f2a43,NaN,solid_slug,idea,"I found this page at random, oddly, and it has..."
2,https://www.halfbakery.com/idea/F_fcrst_20annu...,8e484517-6281-4da1-baf0-9bfc2aec45be,NaN,8th of 7,idea,It seems that the task of detecting and loggin...
3,https://www.halfbakery.com/idea/Plaid_20conduc...,7ca85f88-8662-4c8a-b970-5aa87696c4e3,NaN,beanangel,idea,My Phone's -sometimes-right-side-up- USB mini ...
4,https://www.halfbakery.com/idea/Merry_20Christmas,cba958e5-13fa-485b-bb9a-8c1fb872fd0e,NaN,hippo,idea,Season's greetings to all HalfBakery patrons w...
5,https://www.halfbakery.com/idea/Autoweeebile_20II,42742c2a-aa0d-4f46-ac2b-519b8e1e945a,NaN,beanangel,idea,"[Ixnaum] says, ""you are driving the 2005 Autow..."
6,https://www.halfbakery.com/idea/10_20Gbps_20et...,b5f8a8ea-c334-4000-b533-78b02a990019,NaN,ixnaum,idea,"While 10 Gbps networking is getting cheaper, i..."
7,https://www.halfbakery.com/idea/Completely_20R...,06c313ae-871e-494a-b0f5-70153e4c3b2b,NaN,doctorremulac3,idea,Fake candles with flickering LED faux flames h...
8,https://www.halfbakery.com/idea/Colouring_20Co...,c4c71029-f1e6-4d30-819a-0ba5d70dd406,NaN,2 fries shy of a happy meal,idea,My wife added colouring book to her stocking w...
9,https://www.halfbakery.com/idea/epigenetic_20e...,cc5c4105-626a-4a4d-9c26-39a10b8bec30,NaN,beanangel,idea,More and more people are learning about epigen...


ValueError: DataFrame constructor not properly called!